In [1]:
import os

INPUT_DIR = "../data/nat-ai/text/"
filenames = os.listdir(INPUT_DIR)

texts = []  # list of documents
for filename in filenames:
    with open(INPUT_DIR+filename) as f:
        texts.append(f.read())  # read the whole document

len(texts)

19

In [2]:
from stopwordsiso import stopwords

stopwords = stopwords("en")

more_stopwords = {
    'affect', 'change', 'doe', 'greeting', 'ha', 'herse', 'himse', 'itse', 'key', 'le', 'myse', 'regard', 'result', 'sha', 'ti', 'wa', 'wo', 'word'
}
stopwords = stopwords.union(more_stopwords)

must_include_words = {
    'ai', 'eu'
}
stopwords = stopwords - must_include_words

len(stopwords)

1315

In [3]:
"ai" in stopwords

False

In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/arthit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/arthit/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/arthit/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer

compounds = [
    "artificial intelligence",
    "european union",
    "european parliament",
    "national strategy",
    "government data",
    "public data",
    "public sector",
    "action plan",
]
# compound_tokens = [w.split() for w in compounds]

abbreviations = {
    "ai": "artificial intelligence",
    "eu": "european union",
}

exclude_words = {
    "artificial intelligence",
}

def merge_tokens(tokens, compounds):
    tokens_len = len(tokens)
    i = 0
    while i < tokens_len-1:
        if tokens[i] + " " + tokens[i+1] in compounds:
            new_token = tokens[i] + " " + tokens[i+1]
            new_tokens = tokens[:i] + [new_token] + tokens[i+2:]
            tokens = new_tokens
            tokens_len = len(tokens)
            i = i + 1
        i = i + 1
    return tokens

def replace_tokens(tokens, abbreviations):
    return [abbreviations[t] if t in abbreviations else t for t in tokens]

def exclude_tokens(tokens, exclude_words):
    return [t for t in tokens if t not in exclude_words]

class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        tokens = [self.wnl.lemmatize(t) for t in word_tokenize(doc) if (t.isalpha() and len(t) >= 2)]
        tokens = merge_tokens(tokens, compounds)
        tokens = replace_tokens(tokens, abbreviations)
        tokens = exclude_tokens(tokens, exclude_words)
        return tokens

bow_vectorizer = CountVectorizer(stop_words=stopwords, tokenizer=LemmaTokenizer(), lowercase=True, min_df=10)
bow_matrix = bow_vectorizer.fit_transform(texts)

bow_matrix.shape

(19, 785)

In [6]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

lda_bow  = LDA(n_components=10, random_state=42) 
lda_bow.fit(bow_matrix)

LatentDirichletAllocation(random_state=42)

In [7]:
lda_bow.transform(bow_matrix[:2])

array([[1.43844445e-05, 3.96437387e-03, 1.43828786e-05, 8.69820043e-01,
        1.43838037e-05, 1.15621691e-01, 1.47196509e-03, 1.43843620e-05,
        1.43802141e-05, 9.05001140e-03],
       [5.19903692e-05, 5.19893818e-05, 5.19827735e-05, 1.30193483e-01,
        5.19881678e-05, 1.41779764e-01, 1.35881863e-02, 6.28401008e-01,
        5.19750572e-05, 8.57776334e-02]])

In [8]:
for idx, topic in enumerate(lda_bow.components_):
    print(f"Top 5 words in Topic #{idx+1}:")
    print([bow_vectorizer.get_feature_names()[i] for i in topic.argsort()[-5:]]) 
    print("")

Top 5 words in Topic #1:
['public', 'technology', 'programme', 'government', 'data']

Top 5 words in Topic #2:
['public', 'data', 'support', 'measure', 'digital']

Top 5 words in Topic #3:
['deep', 'programme', 'learning', 'european union', 'oecd']

Top 5 words in Topic #4:
['support', 'technology', 'strategy', 'data', 'development']

Top 5 words in Topic #5:
['develop', 'benefit', 'approach', 'national', 'data']

Top 5 words in Topic #6:
['sector', 'development', 'digital', 'public', 'data']

Top 5 words in Topic #7:
['development', 'business', 'technology', 'government', 'data']

Top 5 words in Topic #8:
['national', 'data', 'vision', 'technology', 'strategy']

Top 5 words in Topic #9:
['procedure', 'base', 'monitor', 'automotive', 'coordinate']

Top 5 words in Topic #10:
['company', 'data', 'development', 'application', 'ministry']



/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
import pyLDAvis 
import pyLDAvis.sklearn 

pyLDAvis.enable_notebook()

/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p17 = _nlv < LooseVersion("1.17")
/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p18 = _nlv < LooseVersion("1.18")
/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py:13: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _np_version_under1p19 = _nlv < LooseVersion("1.19")
/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py:14: DeprecationWarning: distutils Version classes are deprecat

In [10]:
pyLDAvis.sklearn.prepare(lda_bow, bow_matrix, bow_vectorizer)

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.008880  0.002971       1        1  23.099960
9     -0.027223  0.004670       2        1  18.174983
1     -0.050635 -0.086022       3        1  16.832831
3     -0.042318 -0.057383       4        1  13.553530
6     -0.067990  0.005972       5        1  12.665870
7     -0.047556 -0.035343       6        1   8.005806
0     -0.040675  0.000380       7        1   6.697463
4     -0.062104  0.157001       8        1   0.858779
2      0.270082  0.006541       9        1   0.108836
8      0.077300  0.001214      10        1   0.001942, topic_info=               Term         Freq        Total Category  logprob  loglift
284  european union   482.000000   482.000000  Default  30.0000  30.0000
577       programme   631.000000   631.000000  Default  29.0000  29.0000
444        learning   378.000000   378.000000  Default  28.0000  28.0000
192            data  2517.000000  2517.000000  Default  27.0000  27.0000
342      government   898.000000   898.000000  Default  26.0000  26.0000
..              ...          ...          ...      ...      ...      ...
98          capital     0.002417    82.711655  Topic10  -6.6657   0.4085
135      competence     0.002417   218.800715  Topic10  -6.6657  -0.5643
53     architecture     0.002417    23.919373  Topic10  -6.6657   1.6491
349        hardware     0.002417    36.052652  Topic10  -6.6657   1.2388
642           robot     0.002417    63.119727  Topic10  -6.6657   0.6788

[626 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
5         1  0.214077  accelerating
5         2  0.214077  accelerating
5         4  0.321116  accelerating
5         5  0.107039  accelerating
5         7  0.053519  accelerating
...     ...       ...           ...
783       1  0.185137     workforce
783       2  0.143995     workforce
783       4  0.102854     workforce
783       5  0.205707     workforce
783       6  0.390844     workforce

[2508 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 10, 2, 4, 7, 8, 1, 5, 3, 9])